# Исследование надёжности заёмщиков

Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Изучение общей информации

In [1]:
pip install pymystem3 # Установка библиотеки

Note: you may need to restart the kernel to use updated packages.


In [2]:
# импорт библиотек
import pandas as pd
from pymystem3 import Mystem

In [4]:
data = pd.read_csv('\data.csv') 
data.info() 
data.head(10) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Итак, в таблице 12 столбцов. Тип данных  — int64, float64, object.

Согласно документации к данным:

*    `children` — количество детей в семье
*    `days_employed` — общий трудовой стаж в днях
*    `dob_years` — возраст клиента в годах
*    `education` — уровень образования клиента
*    `education_id` — идентификатор уровня образования
*    `family_status` — семейное положение
*    `family_status_id` — идентификатор семейного положения
*    `gender` — пол клиента
*    `income_type` — тип занятости
*    `debt` — имел ли задолженность по возврату кредитов
*    `total_income` — ежемесячный доход
*    `purpose` — цель получения кредита

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

В данных замечен верхний регистр, отрицательные значения стажа, а так же слишком большие значения стажа.

**Вывод**

В каждой строке таблицы — данные об одном клиенте.

Предварительно можно утверждать, что, данных достаточно для проведения исследования. Но встречаются пропуски в данных, нарушения регистра, отрицательные и слишком большие значения в колонке стажа.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Предобработка данных

### Обработка пропусков

In [5]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Больше 2000 пропусков в колонках стажа и ежемесячного дохода, это почти 9%. Несмотря на то, что процент пропусков не большой, удалять строки не стоит, это может исказить конечное исследование.

Так как значения в колонках "стаж" и "ежемесячный доход" являются количественными переменными их нужно заменить на среднее арифметическое и медианное соответственно. Потому что стаж находится в пределах возраста человека, а зарплата может сильно отличаться. Но для начала нужно убрать ошибки в данных.

Необходимо избавиться от аномально больших значений в стаже, возможно стаж указан не в днях, а в часах. Затем уберем отрицательные значения. И в конце заполним пропуски средними и медианными значениями соответсвенно.

In [7]:
def hours_to_days(hours_employed):    
    if hours_employed > 20000:        
        hours_employed /= 24          
    return hours_employed

In [8]:
data['days_employed'] = data['days_employed'].apply(hours_to_days) 

data['days_employed'] = abs(data['days_employed']) 
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].mean()) 
data['total_income'] = data['total_income'].fillna(data['total_income'].median()) 

#проверка
print(data.isna().sum()) 
data.head() 

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [9]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Нужно убрать отрицательные значения. А так же аномальное количество. 

In [10]:
data['children'] = abs(data['children'])
data.loc[data['children'] == 20, 'children'] = 2
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

**Вывод**

Все пропуски устранены. 
Пропуски в данных возможно вызваны техническим фактором, в колонках о доходе и стаже количество пропусков совпадают.
Отрицательные значения в колонке стажа так же могут быть вызваны техническим фактором, разницу взяли из меньшего значения. В то время как аномально большие значения были положительными. Возможно, если действительно такие данные указаны в часах, была какая то ошибка с исходными показателями.

Были найдены аномальные значения в колонке с детьми. Такое количество детей у одно семьи возможно, но не у 76 из 20000. Это скорее уникальный случай. Плюсом в ошибке данных можно считать, что число выбивается из ряда, отсутвуют семьи от 6 до 19 детей. Можно смело утверждать, что в данных была ошибка.

### Замена типа данных

In [12]:
data['days_employed'] = data['days_employed'].astype('int64') 
data['total_income'] = data['total_income'].astype('int64') 
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Осталось всего 2 типа данных, целые числа и строки.
Я использовал метод .astype(), потому что он удобен для замены на нужный тип данных. Целые числа приятны глазу и удобны в работе.

### Обработка дубликатов

In [13]:
data['education'].sort_values().unique()

array(['ВЫСШЕЕ', 'Высшее', 'НАЧАЛЬНОЕ', 'НЕОКОНЧЕННОЕ ВЫСШЕЕ',
       'Начальное', 'Неоконченное высшее', 'СРЕДНЕЕ', 'Среднее',
       'УЧЕНАЯ СТЕПЕНЬ', 'Ученая степень', 'высшее', 'начальное',
       'неоконченное высшее', 'среднее', 'ученая степень'], dtype=object)

In [16]:
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()
print(data['education'].sort_values().unique())
data.duplicated().sum()

['высшее' 'начальное' 'неоконченное высшее' 'среднее' 'ученая степень']


71

In [17]:
data = data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()

0

**Вывод**

Были замечены нарушения регистра в колонке образования, что повлияло на создание неявных дубликатов. Я привел строки к нижнему регистру с помощью метода str.lower(). Затем выявил явные дубликаты с помощью метода duplicated().sum() и удалил их с удалением старых индексов и формированием новых.

### Лемматизация

In [18]:
m = Mystem()

Installing mystem to C:\Users\<user>/.local/bin\mystem.exe from http://download.cdn.yandex.net/mystem/mystem-3.1-win-64bit.zip


In [20]:
# объединим список в строку и удалим лишние символы, возвращает лемматизированные слова
def lemmatizer(column):
    return ''.join(m.lemmatize(column)).strip()

In [21]:
data['purpose'] = data['purpose'].apply(lemmatizer)
data['purpose'].value_counts()

автомобиль                                972
свадьба                                   791
на проведение свадьба                     768
сыграть свадьба                           765
операция с недвижимость                   675
покупка коммерческий недвижимость         661
операция с жилье                          652
покупка жилье для сдача                   651
операция с коммерческий недвижимость      650
покупка жилье                             646
жилье                                     646
покупка жилье для семья                   638
строительство собственный недвижимость    635
недвижимость                              633
операция со свой недвижимость             627
строительство жилой недвижимость          624
покупка недвижимость                      621
покупка свой жилье                        620
строительство недвижимость                619
ремонт жилье                              607
покупка жилой недвижимость                606
на покупка свой автомобиль        

**Вывод**

Перед тем как категоризировать данные, необходимо сначала привести слова к его словарной форме (лемме). В столбце с целью взятия кредита очень много вариантов записи данных. Я применил библиотеку pymystem3 к столбцу 'purpose'. Убрал лишние символы, преобразовал список в строку, теперь с этими данными удобно работать.

### Категоризация данных

In [22]:
def purpose_group(purpose):
    if 'автомобиль' in purpose:
        return 'автомобиль'
    if 'свадьба' in purpose:
        return 'свадьба'
    if 'коммерческий недвижимость' in purpose:
        return 'коммерческая недвижимость'
    if 'строительство' in purpose:
        return 'строительство недвижимости'
    if 'недвижимость' in purpose:
        return 'недвижимость'
    if 'жилье' in purpose:
        return 'недвижимость'
    if 'образование' in purpose:
        return 'образование'
data['purpose_group'] = data['purpose'].apply(purpose_group)
data['purpose_group'].value_counts()

недвижимость                  7622
автомобиль                    4306
образование                   4013
свадьба                       2324
строительство недвижимости    1878
коммерческая недвижимость     1311
Name: purpose_group, dtype: int64

Я разделил людей на 3 условных группы: низкий, средний и высокий. За основу взял медианное значение уровня дохода.

In [23]:
total_income_group = pd.cut(data['total_income'], [0,100000,200000,10000000])
data['total_income'].median()

145017.0

In [24]:
total_income_group.value_counts()

(100000, 200000]      11925
(200000, 10000000]     5066
(0, 100000]            4463
Name: total_income, dtype: int64

**Вывод**

Из всего списка целей по кредиту я выбрал 6 категорий. Значения я внес в добавленный столбец в датафрейме. А так же разделил людей по уровню дохода. Теперь можно будет легко ответить на поставленные вопросы про цели взятия кредита. 

## Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Создам сводную таблицу, сгруппированную по количеству детей. Столбцы: общее количество людей, количество не вернувших кредит в срок, процентное соотношение не вернувших кредит в срок.

In [25]:
data.pivot_table(index=['children'], values ='debt', aggfunc=['count','sum','mean'], margins=True)

,count,sum,mean
,debt,debt,debt
children,,,
0,14091,1063,0.075438
1,4855,445,0.091658
2,2128,202,0.094925
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000
All,21454,1741,0.081150


**Вывод**

Общее количество бездетных людей почти в 2 раза больше. 

Разница людей с детьми не вернувших кредит в срок между людьми без детей в среднем 1,5%. Это говорит о том что зависимость между наличием детей и возвратом кредита в срок крайне незначительная. Не смотря на то, что бездетных людей не вернувших кредит в срок больше, в процентом соотношении они уступают людям с детьми. При увеличении количества людей в исследовании этот показатель может вырасти. Все люди с 5 детьми возвращают кредит в срок.

Можно сделать вывод, что есть небольшая зависимость. Люди с детьми чаще не возвращают кредит в срок. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Создам сводную таблицу, сгруппированную по семейному положению. Столбцы: общее количество людей, количество не вернувших кредит в срок, процентное соотношение не вернувших кредит в срок.

In [26]:
data.pivot_table(index=['family_status'], values ='debt', aggfunc=['count','sum','mean'], margins=True)

,count,sum,mean
,debt,debt,debt
family_status,,,
в разводе,1195,85,0.071130
вдовец / вдова,959,63,0.065693
гражданский брак,4151,388,0.093471
женат / замужем,12339,931,0.075452
не женат / не замужем,2810,274,0.097509
All,21454,1741,0.081150


**Вывод**

Общее количество людей женат/замужем более чем в 2 раза больше остальных. 

Разница людей не женат/замужем и в гражданском браке не вернувших кредит в срок с людьми женат/замужем в среднем 2,5%. Это говорит о том что зависимость между семейным положением и возвратом кредита в срок незначительная. Не смотря на то, что женат/замужем людей не вернувших кредит в срок больше, в процентом соотношении они уступают людям не женат/замужем и в гражданском браке. При увеличении количества людей в исследовании этот показатель может вырасти.

Можно сделать вывод, что есть небольшая зависимость. Люди не женат/замужем и в гражданском браке чаще не возвращают кредит в срок.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Создам сводную таблицу, сгруппированную по уровню дохода. Столбцы: общее количество людей, количество не вернувших кредит в срок, процентное соотношение не вернувших кредит в срок.

In [27]:
data.pivot_table(index=[total_income_group], values ='debt', aggfunc=['count','sum','mean'], margins=True)

,count,sum,mean
,debt,debt,debt
total_income,,,
"(0, 100000]",4463,354,0.079319
"(100000, 200000]",11925,1029,0.086289
"(200000, 10000000]",5066,358,0.070667
All,21454,1741,0.081150


**Вывод**

Количество людей со средним доходом более чем в 2 раза больше других категорий. 

В процентном соотношении людей со среднем доходом больше по не возврату кредита в срок. Далее идут люди с низким уровнем дохода в процентном соотношении. Не смотря на то, что количество людей не вернувших кредит в срок с низким уровнем дохода почти такое же как и с высоким.

Можно сделать вывод, что люди со средним доходом чаще не возвращают кредит в срок.

- Как разные цели кредита влияют на его возврат в срок?

Создам сводную таблицу, сгруппированную по цели взятия кредита. Столбцы: общее количество людей, количество не вернувших кредит в срок, процентное соотношение не вернувших кредит в срок.

In [28]:
data.pivot_table(index=['purpose_group'], values ='debt', aggfunc=['count','sum','mean'], margins=True)

,count,sum,mean
,debt,debt,debt
purpose_group,,,
автомобиль,4306,403,0.093590
коммерческая недвижимость,1311,99,0.075515
недвижимость,7622,539,0.070716
образование,4013,370,0.092200
свадьба,2324,186,0.080034
строительство недвижимости,1878,144,0.076677
All,21454,1741,0.081150


**Вывод**

Больше всего людей берут кредит на собственную недвижимость. 

Выделяются 2 группы, взявшие кредит на автомобиль и образование. В процентном соотношении за автомобиль и образование чаще не возвращают. Не смотря на то, что по количеству невернувших кредит людей лидирует собственную недвижимость. Но и берут чаще всего.

Можно сделать вывод, что люди, взявшие кредит на автомобиль и образование чаще не возвращают кредит в срок. Люди, взявшие кредит на собственную недвижимость самые обязательные.

- Главный вопрос исследования: влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок?

In [29]:
# таблица наличия детей и  количество не вернувших кредита в срок
data.pivot_table(index=['children'], values ='debt',columns='family_status',aggfunc='sum')

family_status,в разводе,вдовец / вдова,гражданский брак,женат / замужем,не женат / не замужем
children,,,,,
0,55.0,53.0,229.0,516.0,210.0
1,21.0,7.0,118.0,247.0,52.0
2,8.0,3.0,33.0,148.0,10.0
3,1.0,0.0,8.0,17.0,1.0
4,0.0,0.0,0.0,3.0,1.0
5,NaN,NaN,0.0,0.0,NaN


In [30]:
# таблица наличия детей и  количество не вернувших кредита в срок
data.pivot_table(index=['children'], values ='debt',columns='family_status',margins=True)

family_status,в разводе,вдовец / вдова,гражданский брак,женат / замужем,не женат / не замужем,All
children,,,,,,
0,0.070153,0.062574,0.083883,0.069095,0.092838,0.075438
1,0.066456,0.086420,0.118000,0.082224,0.114537,0.091658
2,0.096386,0.125000,0.092958,0.093552,0.119048,0.094925
3,0.090909,0.000000,0.142857,0.068273,0.125000,0.081818
4,0.000000,0.000000,0.000000,0.103448,0.500000,0.097561
5,NaN,NaN,0.000000,0.000000,NaN,0.000000
All,0.071130,0.065693,0.093471,0.075452,0.097509,0.081150


**Вывод**

Больше 10% людей не вернувших кредит в срок по отношению к вернувшим, это люди не женат / не замужем и в гражданском браке с детьми от 1 до 3. 

Отдельно стоит отметить вдовец / вдова с 2 детьми 12,5% невозврата. И не женат / не замужем с 4 детьми 50%.

Можно смело дедать вывод, что семейное положение влияет на возврат кредита в срок.

## Общий вывод

Я ответил на 4 вопроса и установил:

1. Люди с детьми чаще не возвращают кредит в срок.
2. Люди живущие в гражданском браке и не женатые/замужние чаще не возвращают кредит в срок.
3. Люди со средним доходом чаще не возвращают кредит в срок.
4. Разные цели кредита влияют на его возврат в срок. Люди, взявшие кредит на автомобиль и образование чаще не возвращают кредит в срок. Люди, взявшие кредит на собственную недвижимость самые обязательные.

**Главный вопрос исследования:** Влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок?
Да, влияет. 

 - Самый большой процент неплательщиков: не женат / не замужем с 4 детьми. Половина не возвращают кредит в срок. 50%
 - На втором месте в гражданском браке с 3 детьми. 14,2%
 - И на третьем месте не женат / не замужем с детьми от 1 до 3 и в гражданском браке с детьми от 1 до 3. Больше 10%
